In [2]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from detector import Detector

In [3]:
def preprocessing(image):
    show(image, 'Origin image')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    show(gray, 'gray')
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    edged = cv2.Canny(gray, 70, 100)
    show(edged, 'edged')

    # define a (3, 3) structuring element
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

    # apply the dilation operation to the edged image
    dilate = cv2.dilate(edged, kernel, iterations=2) # init 1
    show(dilate, 'dilate')
    return dilate
def find_contours(image, dilate):
    # contours detection
    contours,hierarchy = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    image_4_plot = image.copy()
    areas = []
    for cnt in contours:
        # Get the 4 points of the bounding rectangle
        x, y, w, h = cv2.boundingRect(cnt)
        if w*h>10000:
            # Draw a straight rectangle with the points
            img_box = cv2.rectangle(image_4_plot, (x, y), (x+w, y+h), color = (255, 0, 0), thickness = 2)
            areas.append([x, y, w, h])
    show(image_4_plot, 'Contours')
    return image_4_plot, contours, areas

def crop(image, areas):
    mats = []
    crop_offset = 3
    for area in areas:
        # Cropping an image
        x, y, w, h = area
        if y <= crop_offset:
            cropped_image = image[y:(y+h+crop_offset), x-crop_offset:(x+w+crop_offset)]
        else:
            cropped_image = image[y-crop_offset:(y+h+crop_offset), x-crop_offset:(x+w+crop_offset)]
        mats.append(cropped_image)
        # show(cropped_image, 'Crop')
    return mats
# cv2.imwrite(filename_4_saving, image_4_plot)

In [73]:
# Get the path to the 'scans' directory using the current file's location
script_directory = os.getcwd()
scans_directory = os.path.join(script_directory, '..', 'data/scans')
output_directory = os.path.join(script_directory, '..', 'data/outputs')

# Specify the path to your .tif file
# file = 'scan0157'
file = '4bl'
tif_file_path = scans_directory + "/" + f"{file}.tif"
filename_4_saving = output_directory + "/" + f"{file}.png"

# Read the .tif file
image = cv2.imread(tif_file_path, cv2.IMREAD_UNCHANGED)
MIN_AREA = 2000
MAX_AREA = 1000000
 
scale_factor = {'x': 1, 'y': 1}
# image = cv2.resize(image, None, fx=scale_factor['x'], fy=scale_factor['y'])
grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def show(image, name='Result'):
    cv2.namedWindow(name, cv2.WINDOW_NORMAL)
    cv2.imshow(name, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [78]:
if 'g' in file:
    # GREEN
    hsv_min = np.array((56, 66, 10), np.uint8)
    hsv_max = np.array((100, 255, 188), np.uint8)
elif 'bl' in file:
    # black
    hsv_min = np.array((0, 0, 17), np.uint8)
    hsv_max = np.array((179, 115, 76), np.uint8)
elif 'b' in file:
    # BLUE
    hsv_min = np.array((28, 160, 73), np.uint8)
    hsv_max = np.array((139, 255, 255), np.uint8)
elif 'r' in file:
    # RED
    hsv_min = np.array((0, 166, 27), np.uint8)
    hsv_max = np.array((180, 255, 255), np.uint8)
else:
    # white
    hsv_min = np.array((0, 0, 221), np.uint8) # 0, 0, 221
    hsv_max = np.array((139, 11, 255), np.uint8) # 139, 11, 255

In [79]:
show(image, 'Original')
# define the alpha and beta
# alpha = 0.92 # Contrast control
# beta = 10 # Brightness control
#
# # call convertScaleAbs function
# adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV )
thresh = cv2.inRange(hsv, hsv_min, hsv_max)
binary_thresh = thresh / 255
show(thresh, 'thresh')

# Create a blank mask of the same size as the input image
mask = np.zeros_like(thresh)

# Create an empty mask
inverted_thresh = cv2.bitwise_not(thresh)

show(inverted_thresh, 'inverted_thresh')

# Find contours in the mask
mask_contours, _ = cv2.findContours(inverted_thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# Create an output image (initially a copy of the original image)
output_image = thresh.copy()
img_copy = image.copy()
crop_offset = 3
cropped_images = []
# Loop over the contours
for contour in mask_contours:
    area = cv2.contourArea(contour)
    if 200000 < area:
        inverted_thresh_copy = inverted_thresh.copy()
        thresh_copy = thresh.copy()

        # Detect the convex contour
        hull = cv2.convexHull(contour)
        img_hull = cv2.drawContours(img_copy, contours = [hull],
                                contourIdx = 0,
                                color = (255, 0, 0), thickness = 2)
        # show(img_hull, 'img_hull')
        # Convert hull points to the correct format for cv2.fillPoly
        hull_points = np.array(hull, dtype=np.int32)

        # Draw a filled polygon on the mask
        cv2.fillPoly(inverted_thresh_copy, [hull_points], 255)
        cv2.fillPoly(thresh_copy, [hull_points], 255)

        # Apply the mask to the output to retain pixels outside the hull
        final_mask = cv2.bitwise_and(thresh_copy, inverted_thresh_copy)

        # Create a white image of the same size as the original image
        white_image = np.ones_like(img_copy) * 255

        # Apply the mask: where mask is 0, use white_image; where mask is 1, use the original image
        masked_image = np.where(final_mask[:, :, np.newaxis] == 0, white_image, image)

        # Cropping an image
        x, y, w, h = cv2.boundingRect(contour)
        if y <= crop_offset and x <= crop_offset:
            cropped_image = masked_image[y:(y+h+crop_offset), x:(x+w+crop_offset)]
        elif y <= crop_offset:
            cropped_image = masked_image[y:(y+h+crop_offset), x-crop_offset:(x+w+crop_offset)]
        elif x <= crop_offset:
            cropped_image = masked_image[y-crop_offset:(y+h+crop_offset), x:(x+w+crop_offset)]
        else:
            cropped_image = masked_image[y-crop_offset:(y+h+crop_offset), x-crop_offset:(x+w+crop_offset)]

        cropped_images.append(cropped_image)
        show(cropped_image, 'cropped_image')

        # cv2.bitwise_and(thresh_copy, thresh_copy, mask=mask)
        # show(masked_image, 'final_mask')

# # Create a white image of the same size as the original image
# white_image = np.ones_like(image) * 255
#
# # Apply the mask: where mask is 0, use white_image; where mask is 1, use the original image
# masked_image_color = np.where(thresh[:, :, np.newaxis] == 255, white_image, image)
# show(masked_image_color, 'masked_image')

In [33]:
# define the alpha and beta
alpha = 0.95 # Contrast control
beta = 20 # Brightness control

# call convertScaleAbs function
adjusted = cv2.convertScaleAbs(image, alpha=alpha)
show(adjusted, 'adjusted')


In [38]:

# Create a white image of the same size as the original image
white_image = np.ones_like(image) * 255

# Apply the mask: where mask is 0, use white_image; where mask is 1, use the original image
masked_image_color = np.where(thresh[:, :, np.newaxis] == 255, white_image, image)
show(masked_image_color, 'masked_image')

dilate = preprocessing(masked_image_color)
img, contours, areas = find_contours(masked_image_color, dilate)
img_copy = masked_image_color.copy()
crop_offset = 3
cropped_images = []
for cnt in contours:
    area = cv2.contourArea(cnt)
    if area > 100000:
        # Detect the convex contour
        hull = cv2.convexHull(cnt)
        img_hull = cv2.drawContours(img_copy, contours = [hull],
                                    contourIdx = 0,
                                    color = (255, 0, 0), thickness = 2)
        # Created a new mask and used bitwise_and to select for contours:
        mask = np.ones_like(img_copy[:, :, 0], dtype=np.uint8)
        # mask = np.zeros_like(img_copy)
        cv2.drawContours(mask, [hull], 0, (255, 255, 255), -1)
        # show(img_hull, 'MASK')

        # Create a white image of the same size as the original image
        white_image = np.ones_like(img_copy) * 255

        # Apply the mask: where mask is 0, use white_image; where mask is 1, use the original image
        masked_image = np.where(mask[:, :, np.newaxis] == 1, white_image, masked_image_color)

        # Cropping an image
        x, y, w, h = cv2.boundingRect(cnt)
        if y <= crop_offset:
            cropped_image = masked_image[y:(y+h+crop_offset), x-crop_offset:(x+w+crop_offset)]
        else:
            cropped_image = masked_image[y-crop_offset:(y+h+crop_offset), x-crop_offset:(x+w+crop_offset)]

        cropped_images.append(cropped_image)
        show(cropped_image)
# show(img_copy)

In [13]:
dilate1 = preprocessing(cropped_images[0])
img1, contours1, areas1 = find_contours(cropped_images[0], dilate1)
# show(img1)

img1_copy = img1.copy()

for cnt in contours1:
    area = cv2.contourArea(cnt)
    if area > 1000:
        # Detect the convex contour
        hull = cv2.convexHull(cnt)
        img_hull = cv2.drawContours(img1_copy, contours = [hull],
                                    contourIdx = 0,
                                    color = (255, 0, 0), thickness = 2)
        # Created a new mask and used bitwise_and to select for contours:
        # mask = np.ones_like(img1_copy[:, :, 0], dtype=np.uint8)
        # cv2.drawContours(mask, [hull], -1, 0, -1)
        #
        # # Create a white image of the same size as the original image
        # white_image = np.ones_like(img1) * 255
        #
        # # Apply the mask: where mask is 0, use white_image; where mask is 1, use the original image
        # masked_image = np.where(mask[:, :, np.newaxis] == 1, white_image, img1)
        #
        # # Cropping an image
        # x, y, w, h = cv2.boundingRect(cnt)
        # cropped_image = masked_image[y-crop_offset:(y+h+crop_offset), x-crop_offset:(x+w+crop_offset)]
        # cropped_images.append(cropped_image)
        show(img_hull)

IndexError: list index out of range

In [ ]:
cv2.contorArea(contours)

In [26]:
# contours = max(contours, key = lambda x: cv2.contourArea(x))
# Created a new mask and used bitwise_and to select for contours:
hull = cv2.convexHull(contours)
mask = np.zeros_like(image)
cv2.drawContours(mask, [hull], -1, (255, 255, 255), -1)
masked_image = cv2.bitwise_and(image, mask)
show(masked_image)

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'convexHull'
> Overload resolution failed:
>  - points is not a numerical tuple
>  - Expected Ptr<cv::UMat> for argument 'points'


In [25]:
# ret , thrash = cv2.threshold(grayImage, 226 , 255, cv2.CHAIN_APPROX_NONE)
contours , hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
image_4_plot = image.copy()
for contour in contours:
    area = cv2.contourArea(contour)
    if area>150000:
        print(area)
        approx = cv2.approxPolyDP(contour, 0.01* cv2.arcLength(contour, True), True)
        cv2.drawContours(image_4_plot, [approx], 0, (255, 0, 0), 2)
        x = approx.ravel()[0]
        y = approx.ravel()[1] - 5

        if len(approx) == 3:
            cv2.putText(image_4_plot, "Triangle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0) )
        elif len(approx) == 4 :
            x, y , w, h = cv2.boundingRect(approx)
            aspectRatio = float(w)/h
            print(aspectRatio)
            if aspectRatio >= 0.95 and aspectRatio < 1.05:
                cv2.putText(image_4_plot, "square", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))

            else:
                cv2.putText(image_4_plot, "rectangle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))

        elif len(approx) == 5 :
            cv2.putText(image_4_plot, "pentagon", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
        elif len(approx) == 10 :
            cv2.putText(image_4_plot, "star", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
        else:
            cv2.putText(image_4_plot, "circle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))

cv2.imshow('shapes', image_4_plot)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [28]:
def nothing(x):pass


cv2.namedWindow('videoUI', cv2.WINDOW_NORMAL)
cv2.createTrackbar('T','videoUI',0,255,nothing)

while(True):
    # vid_gray = cv2.cvtColor(grayImage, cv2.COLOR_BGR2GRAY)
    thresh = cv2.getTrackbarPos('T','videoUI')
    ret,thresh = cv2.threshold(gray,thresh,255,0)  #to detect white objects
    #to get outer boundery only
    kernel = np.ones((2,2),np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_GRADIENT, kernel)
    #to strength week pixels
    thresh = cv2.dilate(thresh,kernel,iterations = 5)
    # vid_bw = cv2.threshold(grayImage, thresh, 255, cv2.THRESH_BINARY)[1]

    cv2.imshow('videoUI',cv2.flip(thresh,1))

    if cv2.waitKey(0) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break
